<a href="https://colab.research.google.com/github/Diwakarmogarala/Capstone/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import joblib
from google.colab import drive
drive.mount('/content/drive')
import pandas as pl

Mounted at /content/drive


In [36]:
X = pl.read_csv("/content/drive/MyDrive/Capstone Project/X.csv")
y = pl.read_csv("/content/drive/MyDrive/Capstone Project/y.csv")

In [3]:
Z = pl.concat([X,y], axis = 1)
Z = Z.drop(columns=['Ethnicity', "Payment Typology 3","Hospital County", "Year"], axis=1)
Z

,Age Group,Gender,Race,Type of Admission,Patient Disposition,APR DRG Description,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Total Charges
0,4,0,3,5,8,463,15,3,3,0,5,8,3
1,2,1,3,0,8,463,15,2,2,0,8,7,2
2,3,0,3,5,8,90,15,2,2,0,5,2,7
3,4,1,3,5,8,463,15,3,3,0,5,4,4
4,3,1,3,0,8,463,15,3,3,0,0,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15670363,1,1,2,1,8,203,4,0,1,0,4,7,125
15670364,3,1,2,1,8,544,21,1,1,0,4,7,21
15670365,3,1,2,1,18,285,9,3,2,0,4,7,3
15670366,2,0,2,1,18,22,9,1,3,0,4,7,8


In [29]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
metrics ={"Group":[], "MSE":[], "R2":[]}
for i in Z["APR MDC Description"].unique():
    Z_mod = Z[Z["APR MDC Description"]==i]
    x = Z_mod.drop(columns=["Total Charges"], axis=1)
    y = Z_mod["Total Charges"]
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)
    from sklearn.neural_network import MLPRegressor
    model = XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', n_estimators=100, random_state=42)
    model.fit(x_train, y_train)


    from sklearn.metrics import mean_squared_error, r2_score
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    metrics["Group"].append(i)
    metrics["MSE"].append(mse)
    metrics["R2"].append(r2)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:47:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:47:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:2676: UserWarning: [05:47:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183:

In [30]:
data = pl.DataFrame(metrics)
data = data.sort_values(by="Group", ascending=True)
data = data[data["Group"]<34]
data = data.reset_index()
data = data.drop(columns=["index"], axis=1)
data

,Group,MSE,R2
0,0,29.130890,0.422710
1,1,89.669197,0.490288
2,2,31.784836,0.294978
3,3,35.516129,0.317703
4,4,26.133385,0.456696
5,5,24.735601,0.412269
6,6,21.934822,0.377439
7,7,28.715952,0.084599
8,8,17.326811,0.349872
9,9,30.205942,0.393486


In [31]:
df = pl.read_csv("/content/drive/MyDrive/Capstone Project/merged_data_with_prices.csv")

In [32]:
apr_mdc_description = df["APR MDC Description"].value_counts().reset_index().sort_values(by="APR MDC Description")
apr_mdc_description
apr_mdc_description = apr_mdc_description.reset_index()
del apr_mdc_description["index"]
apr_mdc_description

,APR MDC Description,count
0,alcohol/drug use and alcohol/drug induced orga...,488868
1,burns,13805
2,"diseases and disorders of blood, blood forming...",66860
3,diseases and disorders of the blood and blood ...,152144
4,diseases and disorders of the circulatory system,1960456
5,diseases and disorders of the digestive system,1230397
6,"diseases and disorders of the ear, nose, mouth...",136892
7,diseases and disorders of the eye,24641
8,diseases and disorders of the female reproduct...,142178
9,diseases and disorders of the hepatobiliary sy...,437224


In [33]:
data["Apr MDC Description"] = apr_mdc_description["APR MDC Description"]
data["Count"] = apr_mdc_description["count"]

In [34]:
data

,Group,MSE,R2,Apr MDC Description,Count
0,0,29.130890,0.422710,alcohol/drug use and alcohol/drug induced orga...,488868
1,1,89.669197,0.490288,burns,13805
2,2,31.784836,0.294978,"diseases and disorders of blood, blood forming...",66860
3,3,35.516129,0.317703,diseases and disorders of the blood and blood ...,152144
4,4,26.133385,0.456696,diseases and disorders of the circulatory system,1960456
5,5,24.735601,0.412269,diseases and disorders of the digestive system,1230397
6,6,21.934822,0.377439,"diseases and disorders of the ear, nose, mouth...",136892
7,7,28.715952,0.084599,diseases and disorders of the eye,24641
8,8,17.326811,0.349872,diseases and disorders of the female reproduct...,142178
9,9,30.205942,0.393486,diseases and disorders of the hepatobiliary sy...,437224


In [35]:
data.to_csv("LOS_metrics.csv")

In [37]:
X = pl.concat([X, df["Length of Stay"]],axis = 1)
y= df["Total Charges"]

In [38]:
X

,Year,Hospital County,Age Group,Gender,Race,Ethnicity,Type of Admission,Patient Disposition,APR DRG Description,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Length of Stay
0,2016,1,4,0,3,1,5,8,463,15,3,3,0,5,8,7,3
1,2016,1,2,1,3,1,0,8,463,15,2,2,0,8,7,7,2
2,2016,1,3,0,3,1,5,8,90,15,2,2,0,5,2,7,7
3,2016,1,4,1,3,1,5,8,463,15,3,3,0,5,4,7,4
4,2016,1,3,1,3,1,0,8,463,15,3,3,0,0,7,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15670363,2022,25,1,1,2,2,1,8,203,4,0,1,0,4,7,7,125
15670364,2022,25,3,1,2,3,1,8,544,21,1,1,0,4,7,7,21
15670365,2022,25,3,1,2,3,1,18,285,9,3,2,0,4,7,7,3
15670366,2022,25,2,0,2,2,1,18,22,9,1,3,0,4,7,7,8


In [39]:
Z = pl.concat([X,y], axis = 1)
Z = Z.drop(columns=['Ethnicity', "Payment Typology 3","Hospital County", "Year"], axis=1)
from xgboost import XGBRegressor

model = XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', n_estimators=100, random_state=42)
model.fit(x_train, y_train)

metrics ={"Group":[], "MSE":[], "R2":[]}
for i in Z["APR MDC Description"].unique():
    Z_mod = Z[Z["APR MDC Description"]==i]
    X = Z_mod.drop(columns=["Total Charges"], axis=1)
    y = Z_mod["Total Charges"]
    x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
    from sklearn.neural_network import MLPRegressor
    model = XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', n_estimators=100, random_state=42)
    model.fit(x_train, y_train)


    from sklearn.metrics import mean_squared_error, r2_score
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    metrics["Group"].append(i)
    metrics["MSE"].append(mse)
    metrics["R2"].append(r2)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:50:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:50:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:50:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [05:50:

In [40]:
data = pl.DataFrame(metrics)
data = data.sort_values(by="Group", ascending=True)
data = data[data["Group"]<34]
data = data.reset_index()
data = data.drop(columns=["index"], axis=1)
data

,Group,MSE,R2
0,0,2.161111e+09,0.324925
1,1,4.130464e+10,0.622490
2,2,4.062031e+09,0.603820
3,3,8.411983e+09,0.610766
4,4,6.877288e+09,0.722512
5,5,3.823939e+09,0.622021
6,6,3.629304e+09,0.610331
7,7,2.652568e+09,0.307860
8,8,4.242582e+09,0.379196
9,9,5.545268e+09,0.695349


In [41]:
apr_mdc_description = df["APR MDC Description"].value_counts().reset_index().sort_values(by="APR MDC Description")
apr_mdc_description
apr_mdc_description = apr_mdc_description.reset_index()
del apr_mdc_description["index"]
apr_mdc_description

,APR MDC Description,count
0,alcohol/drug use and alcohol/drug induced orga...,488868
1,burns,13805
2,"diseases and disorders of blood, blood forming...",66860
3,diseases and disorders of the blood and blood ...,152144
4,diseases and disorders of the circulatory system,1960456
5,diseases and disorders of the digestive system,1230397
6,"diseases and disorders of the ear, nose, mouth...",136892
7,diseases and disorders of the eye,24641
8,diseases and disorders of the female reproduct...,142178
9,diseases and disorders of the hepatobiliary sy...,437224


In [42]:
data["Apr MDC Description"] = apr_mdc_description["APR MDC Description"]
data["Count"] = apr_mdc_description["count"]

In [43]:
data

,Group,MSE,R2,Apr MDC Description,Count
0,0,2.161111e+09,0.324925,alcohol/drug use and alcohol/drug induced orga...,488868
1,1,4.130464e+10,0.622490,burns,13805
2,2,4.062031e+09,0.603820,"diseases and disorders of blood, blood forming...",66860
3,3,8.411983e+09,0.610766,diseases and disorders of the blood and blood ...,152144
4,4,6.877288e+09,0.722512,diseases and disorders of the circulatory system,1960456
5,5,3.823939e+09,0.622021,diseases and disorders of the digestive system,1230397
6,6,3.629304e+09,0.610331,"diseases and disorders of the ear, nose, mouth...",136892
7,7,2.652568e+09,0.307860,diseases and disorders of the eye,24641
8,8,4.242582e+09,0.379196,diseases and disorders of the female reproduct...,142178
9,9,5.545268e+09,0.695349,diseases and disorders of the hepatobiliary sy...,437224


In [44]:
data.to_csv("Price_metrics.csv")